In [1]:
# Import libraries
import requests
import re # regular expressions
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep

In [2]:
# local variables
subCategorListTitle  = "j-menu-item"
CHROME_DRIVER_PATH = r'C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe'
URL = 'https://www.wildberries.ru/catalog/muzhchinam/odezhda/bryuki-i-shorty'


driver = webdriver.Chrome(CHROME_DRIVER_PATH)
driver.get(URL)
driver.maximize_window()

content = driver.page_source
soup = BeautifulSoup(content)

prices = []
brands = []
categories = []

data = {
        "Categories" : categories,
        "Brand" : brands,
        "Price" : prices
}

df = pd.DataFrame(data)

In [3]:
#To wait till page loads
def waitTillPageLoads(classNameLoaded, URL_name):
    driver.get(URL_name)
    delay = 10
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, classNameLoaded)))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
        
waitTillPageLoads('lower-price', driver.current_url)


Page is ready!


In [4]:
def searchAllItemsOfCurrentPage(prices, brands, categories):
    class_title = 'j-card-item'
    res = []
    sizes_available = []
    content = driver.page_source
    soup = BeautifulSoup(content)
    for block in soup.findAll('div', attrs={'class':class_title}):
        class_title = 'j-card-item'

        # Get price of clothes
        priceString = "0"
        priceBlock = block.find('ins', attrs={'class': 'lower-price'})
        if(priceBlock != None):
            priceString = block.find('ins', attrs={'class': 'lower-price'}).text
        elif (block.find('span', attrs={'class': 'lower-price'}) == None):
            priceString = "0"
        else:
            priceString = block.find('span', attrs={'class': 'lower-price'}).text
        price = unidecode.unidecode(priceString.strip())
        prices.append(price)
        # Get brand of clothes
        brandString = block.find('strong', attrs={'class': 'brand-name'}).text
        brand = unidecode.unidecode(brandString.strip())
        brands.append(brand)

        # Get category of clothes
        categoryString = block.find('span', attrs={'class': 'goods-name'}).text
        category = unidecode.unidecode(categoryString.strip())
        categories.append(category)
        sizes = []
    res.append(categories)
    res.append(brands)
    res.append(prices)
    addDataToDF(res, df)
        # todo
        # Get sizes of clothes
        
    #    for sizes_block in block.find('span', attrs = {'class': 'product-card__sizes'}):
    #        sizeString = sizes_block.find('a', attrs={'class': 'j-size'}).text
    #        size = unidecode.unidecode(sizeString.strip())
    #        sizes.append(size)
    #        print(sizes)
    #     sizes_available.append(sizes)

In [5]:
def addDataToDF(list, df):
    count = 0
    df2 = pd.DataFrame(data=list).T
    df2.columns = ["Categories", "Brand", "Price"]
    df = pd.concat([df, df2])
    df.to_excel('data.xlsx',index=False)
    print(df)

In [6]:
#buttons
burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")
allButtons = [forMenButton, forWomenButton, shoesButton, forBabiesButton, accessoriesButton]


prices = []
brands = []
categories = []



#Search for shoes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
    ActionChains(driver).click(burgerButton).pause(1).click(shoesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 5:
        break

        
        
        
        
#Search for women clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
    ActionChains(driver).click(burgerButton).pause(1).click(forWomenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 28:
        break
 

#Search for babies clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
    ActionChains(driver).click(burgerButton).pause(1).click(forBabiesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 2:
        break
 
#Search for men clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
    ActionChains(driver).click(burgerButton).pause(1).click(forMenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 26:
        break

#Search for accessories
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")
    ActionChains(driver).click(burgerButton).pause(1).click(accessoriesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 18:
        break



Page is ready!
                                 Categories     Brand    Price
0                                   Botinki    KUMMAN  6 059 R
1                                   Botinki  Antilopa  4 180 R
2                               Botinki tec    Lassie  4 399 R
3                                   Botinki      GEOX  5 392 R
4                                   Botinki      GEOX  5 392 R
..                                      ...       ...      ...
100  Botinki dlia devochki osennie martinsy       KDX  2 099 R
101                    Botinki tec Carlisle    Lassie  3 899 R
102                 Botinki dlia mal'chikov      ELSA  5 568 R
103    Botinki dlia devochki detskie zimnie    Mikasa  2 562 R
104    Zimnie botinki dutiki dlia mal'chika  Blessbox  2 470 R

[105 rows x 3 columns]
Page is ready!
                                            Categories      Brand    Price
0                                              Botinki     KUMMAN  6 059 R
1                                       

Page is ready!
                                            Categories              Brand  \
0                                              Botinki             KUMMAN   
1                                              Botinki           Antilopa   
2                                          Botinki tec             Lassie   
3                                              Botinki               GEOX   
4                                              Botinki               GEOX   
..                                                 ...                ...   
923          Dzhegginsy uteplennye s nachesom na mekhu            Avrilla   
924         Dzhinsy zhenskie s vysokoi posadkoi modnye             Bronks   
925  Dzhinsy MOM dlia beremennykh skinni so vstavko...             Preggy   
926                Dzhinsy zhenskie priamye na rezinke          FEIMAILIS   
927         Dzhegginsy s vysokoi posadskoi uteplionnye  NATASHA BEAUTIFUL   

       Price  
0    6 059 R  
1    4 180 R  
2    4 399 R  


Page is ready!
                                        Categories          Brand    Price
0                                          Botinki         KUMMAN  6 059 R
1                                          Botinki       Antilopa  4 180 R
2                                      Botinki tec         Lassie  4 399 R
3                                          Botinki           GEOX  5 392 R
4                                          Botinki           GEOX  5 392 R
...                                            ...            ...      ...
1851                                        Khalat        Amadel'  1 174 R
1852      Khalat makhrovyi, Khalat zhenskii bannyi  Wakas Textile  1 800 R
1853  Khalat domashnii zhenskii na molnii, khlopok       ROYALTEX  1 249 R
1854  Khalat domashnii zhenskii na molnii, khlopok       ROYALTEX  1 249 R
1855          Khalat zhenskii makhrovyi veliurovyi   TERRY DREAMS  3 360 R

[1856 rows x 3 columns]
Page is ready!
                                      Categor

Page is ready!
                                         Categories           Brand    Price
0                                           Botinki          KUMMAN  6 059 R
1                                           Botinki        Antilopa  4 180 R
2                                       Botinki tec          Lassie  4 399 R
3                                           Botinki            GEOX  5 392 R
4                                           Botinki            GEOX  5 392 R
...                                             ...             ...      ...
2684     Legginsy losiny zhenskie sportivnye pushap       Bona Fide  4 950 R
2685        Longsliv zhenskii s vyrezom kare modnyi        The Croy  2 029 R
2686    Sviter zhenskii viazanyi oversaiz, dzhemper         LUXINNE  3 598 R
2687  Pizhama zhenskaia s shortami bol'shie razmery      MARFAWOMEN  1 469 R
2688       Legginsy teplye zhenskie losiny na mekhu  A.N.K. Stories    512 R

[2689 rows x 3 columns]
Page is ready!
                     

Page is ready!
                                      Categories         Brand     Price
0                                        Botinki        KUMMAN   6 059 R
1                                        Botinki      Antilopa   4 180 R
2                                    Botinki tec        Lassie   4 399 R
3                                        Botinki          GEOX   5 392 R
4                                        Botinki          GEOX   5 392 R
...                                          ...           ...       ...
3523                                  Sapogi tec        Lassie   4 599 R
3524  Kolgotki dlia devochki khlopok nabor 3 sht     Timy love     708 R
3525                                      Pal'to  Gloria Jeans   6 499 R
3526       Zimnii kombinezon iz membrannoi tkani     Arctiline  17 094 R
3527                              Polukombinezon         Gusti   3 742 R

[3528 rows x 3 columns]
Page is ready!
                                         Categories       Brand    Pr

Page is ready!
                                             Categories     Brand    Price
0                                               Botinki    KUMMAN  6 059 R
1                                               Botinki  Antilopa  4 180 R
2                                           Botinki tec    Lassie  4 399 R
3                                               Botinki      GEOX  5 392 R
4                                               Botinki      GEOX  5 392 R
...                                                 ...       ...      ...
4447  Bleizer viazanyi dlia muzhchin Kardigan muzhsk...     VANDA  2 850 R
4448        Muzhskoi klassicheskii pidzhak s podkladkoi   STENSER  7 089 R
4449                                             Zhilet       T&H  2 227 R
4450       Muzhskoi zhilet iz ovech'ei shersti mekhovoi     ALTRO  4 041 R
4451           Rabochaia odezhda muzhskaia siz zhiletka     Fakel  1 812 R

[4452 rows x 3 columns]
Page is ready!
                                          Cat

Page is ready!
                                             Categories          Brand  \
0                                               Botinki         KUMMAN   
1                                               Botinki       Antilopa   
2                                           Botinki tec         Lassie   
3                                               Botinki           GEOX   
4                                               Botinki           GEOX   
...                                                 ...            ...   
5379              Dzhinsy briuki shtany dzhoggery kargo          Banda   
5380  Futbolka muzhskaia belaia khlopok odnotonnaia ...   Maks-Ekstrim   
5381             Briuki priamye muzhskie, klassicheskie          LAINA   
5382             Briuki dzhoggery muzhskie shtany kargo   Barouz Jeans   
5383                     Steganaia Kurtka Atlas Outdoor  Atlas For Men   

        Price  
0     6 059 R  
1     4 180 R  
2     4 399 R  
3     5 392 R  
4     5 392 R  


Page is ready!
                                             Categories          Brand  \
0                                               Botinki         KUMMAN   
1                                               Botinki       Antilopa   
2                                           Botinki tec         Lassie   
3                                               Botinki           GEOX   
4                                               Botinki           GEOX   
...                                                 ...            ...   
6303  Obodok dlia volos zhenskii, obodki dlia devoch...  LUMINA JEWELS   
6304         Parik dlia zhenshchin modnyi gustye volosy       Trtsh&Co   
6305                          Kanekalon dlia volos Aida       HAIRSHOP   
6306                                  krabik dlia volos         Jinero   
6307          Kanekalon dlia volos aksessuar ukrashenie       HAIRSHOP   

        Price  
0     6 059 R  
1     4 180 R  
2     4 399 R  
3     5 392 R  
4     5 392 R  


Page is ready!
                                             Categories           Brand  \
0                                               Botinki          KUMMAN   
1                                               Botinki        Antilopa   
2                                           Botinki tec          Lassie   
3                                               Botinki            GEOX   
4                                               Botinki            GEOX   
...                                                 ...             ...   
7019       Koshelek zhenskii krasnyi natural'naia kozha  Vincenzo Ricci   
7020       Koshelek muzhskoi kozhanyi Portmone muzhskoe       Mimi Mode   
7021  Koshelek muzhskoi iz natural'noi kozhi, portmo...       Mark Well   
7022                Koshelek zhenskii kozhanyi portmone  Cactus.bagshop   
7023       Koshelek muzhskoi kozhanyi tonkii s zazhimom     MOJJO brand   

        Price  
0     6 059 R  
1     4 180 R  
2     4 399 R  
3     5 392 R  
4   

Page is ready!
                                         Categories            Brand    Price
0                                           Botinki           KUMMAN  6 059 R
1                                           Botinki         Antilopa  4 180 R
2                                       Botinki tec           Lassie  4 399 R
3                                           Botinki             GEOX  5 392 R
4                                           Botinki             GEOX  5 392 R
...                                             ...              ...      ...
7738   Poias dlia plat'ia korset zhenskii vechernii            L.A.F    550 R
7739             Remen' muzhskoi natural'naia kozha     Leroy King's  1 027 R
7740   Portupeia dlia zhenshchin 18+, garter, kozha            Rabby  1 714 R
7741  Remen' muzhskoi kozhanyi dlia dzhinsov, briuk            TEZZE  1 605 R
7742                  Remen' kozhanyi klassicheskii  Riccardo Donati    990 R

[7743 rows x 3 columns]
Page is ready!
         